## Question 2

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor as kNNRegression

def pn(x):
    print(x,'\n')
    return

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [55]:
pip install --upgrade scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 100.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: numpy in /sfs/qumulo/qhome/hek4z/.local/lib/python3.8/site-packages (1.24.4)
Note: you may need to restart the kernel to use updated packages.


In [15]:
## Question (1)

df = pd.read_csv('./data/USA_cars_datasets.csv')
print(df.dtypes)
df.head()

Unnamed: 0       int64
price            int64
brand           object
model           object
year             int64
title_status    object
mileage          int64
color           object
vin             object
lot              int64
state           object
country         object
condition       object
dtype: object


,Unnamed: 0,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,0,6300,toyota,cruiser,2008,clean vehicle,274117,black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,1,2899,ford,se,2011,clean vehicle,190552,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,2,5350,dodge,mpv,2018,clean vehicle,39590,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left
3,3,25000,ford,door,2014,clean vehicle,64146,blue,1ftfw1et4efc23745,167753855,virginia,usa,22 hours left
4,4,27700,chevrolet,1500,2018,clean vehicle,6654,red,3gcpcrec2jg473991,167763266,florida,usa,22 hours left


In [16]:
columns = ['price','year','mileage']
cdf = df[columns].copy()

print(cdf.head())
print('\n')
print(cdf.shape)

column_name = 'price'  # price has no NAs
nan_check = cdf[column_name].isna()

# Printing the rows where NaN values are present
print(cdf[nan_check])

column_name = 'year'  # year has no NAs
nan_check = cdf[column_name].isna()

# Printing the rows where NaN values are present
print(cdf[nan_check])

column_name = 'mileage'  # mileage has no NAs
nan_check = cdf[column_name].isna()

# # Printing the rows where NaN values are present
print(cdf[nan_check])


## There are no NAs to handle in this data set

   price  year  mileage
0   6300  2008   274117
1   2899  2011   190552
2   5350  2018    39590
3  25000  2014    64146
4  27700  2018     6654


(2499, 3)
Empty DataFrame
Columns: [price, year, mileage]
Index: []
Empty DataFrame
Columns: [price, year, mileage]
Index: []
Empty DataFrame
Columns: [price, year, mileage]
Index: []


In [33]:
# Question (2)

def maxmin(x): # Max-min scaler function
    x = (x-min(x))/(max(x)-min(x))
    return x

# Preprocess the data:
y = cdf['price'] # Create target variable vector
X = cdf.loc[:,['year','mileage']] # Create feature matrix
X = X.apply(maxmin) # Normalize X

In [39]:
# Question (3)

import matplotlib.pyplot as plt


# If you wanted to split without sk:
N = cdf.shape[0] # Total number of rows
all_rows = np.arange(N) # Create a list of all row indices
train_rows = np.random.choice(N,int(.2*N),replace=False) # Draw 20% of the rows for testing
test_rows = list(set(all_rows)-set(train_rows)) # Find the remaining rows for training

# Create train/test data:
X_train = X.iloc[train_rows,:]
y_train = y.iloc[train_rows]
X_test = X.iloc[test_rows,:]
y_test = y.iloc[train_rows]

In [30]:
# Question (4)

# A simple implementaton of kNN for regression:
def knn_hat(x_hat,cdf,k):
    
    # Compute distances between x_hat and the data:  
    squared_difference = (x_hat - cdf.loc[:,['year','mileage']])**2
    distances = np.sum( squared_difference , axis = 1)
    
    # Find k smallest values in dist:
    neighbors = np.argsort(distances)[:k].tolist()
    
    # Find y values for the nearest neighbors:
    y_star = cdf['y'].iloc[neighbors].tolist()
    
    # Average neighbor values to get prediction:
    y_hat = np.mean(y_star)
    
    # Return a dictionary of computed values of interest:
    return({'y_hat':y_hat, 'y_star':y_star, 'neighbors':neighbors})

In [50]:
def statics(k=3,data=cdf): # This function replicates our previous work, with k as an input
    y = cdf['price'] # Create target variable vector
    X = cdf.loc[:,['year','mileage']] # Create feature matrix
    # Normalize:
    X = X.apply(maxmin)
    model = kNNRegression(n_neighbors=k) # Create a sk model for k=3
    fitted_model = model.fit(X,y) # Train the model on our data
    N_x = 100 # Coarseness of x variable
    N_y = 100 # Coarseness of y variable
    total = N_x*N_y # Total number of points to plot
    grid_x = np.linspace(0,1,N_x) # Create a grid of x values
    grid_y = np.linspace(0,1,N_y) # Create a grid of y values
    xs, ys = np.meshgrid(grid_x,grid_y) # Explode grids to all possible pairs
    X = xs.reshape(total) # Turns pairs into vectors
    Y = ys.reshape(total) # Turns pairs into vectors
    x_hat = pd.DataFrame({'year':X,'mileage':Y}) # Create a dataframe of points to plot
    y_hat = fitted_model.predict(x_hat) # Fit the model to the points
    x_hat['price'] = y_hat # Add new variable to the dataframe
    this_plot = sns.scatterplot(data=x_hat,x='year',y='mileage',hue='price') # Create seaborn plot
    sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1)) # Move legend off the plot canvas

In [53]:
statics(3)

NameError: name 'kNNRegression' is not defined

In [31]:
def pn(x):
    print(x,'\n')
    return

pn(cdf.head())
x_hat=np.array((1,1))
pn(x_hat)
pn(np.sqrt(np.sum))#not finished 

   price  year  mileage
0   6300  2008   274117
1   2899  2011   190552
2   5350  2018    39590
3  25000  2014    64146
4  27700  2018     6654 

[1 1] 



TypeError: loop of ufunc does not support argument 0 of type function which has no callable sqrt method

In [32]:
# Example:
x_hat = np.array((1,1)) # Example prediction case
knn = knn_hat(x_hat, cdf, 3) # Call our function
pn(knn['neighbors'])
pn(knn['y_star'])
pn(knn['y_hat'])


# cdf2 = cdf
# hat_row = [knn['y_hat'], x_hat[0], x_hat[1],'hat']
# cdf2.loc[len(cdf2)] = hat_row
# this_plot = sns.scatterplot(data=gdf2,x='x1',y='x2',hue='y',style='g') # Plot the fake data
# sns.move_legend(this_plot, "upper left", bbox_to_anchor=(1, 1))

KeyError: 'y'

In [35]:
# Question (4)

k_bar = 70
k_grid = np.arange(1,k_bar)
SSE_train = np.zeros(k_bar)
SSE_test = np.zeros(k_bar)
for k in range(k_bar):
    model = kNNRegression(n_neighbors=k+1) # Create a sk model for k
    fitted_model = model.fit(X_train,y_train) # Train the model on our data
    y_hat = fitted_model.predict(X_test) # Predict values for test set
    SSE_test[k] = np.sum( (y_test-y_hat)**2 ) # Save the computed SSE
    y_hat = fitted_model.predict(X_train) # Predict values for training set
    SSE_train[k] = np.sum( (y_train-y_hat)**2 ) # Save the computed SSE

SSE_min = np.min(SSE) # Lowest recorded SSE
min_index = np.where(SSE==SSE_min) # Find the indices of y that equal the minimum
k_star = k_grid[min_index] # Find the optimal value of k
pn(k_star)

NameError: name 'kNNRegression' is not defined